In [ ]:
# Hoje fiz uma automação que atualiza em tempo real o valor da cotação do dólar, euro e ouro e atualiza uma base de dados
# e envia por email em excel


In [53]:
!pip install selenium

#instalar o webdriver
# no chrome instalar o chromedriver

In [54]:
# Selenium é um biblioteca que gera um navegador. usar o from importa apenas algo especifico de qualquer biblioteca.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()




# entrar no google

navegador.get("https://www.google.com/")

#xpath usar aspas simples

# Passo 1: Pegar a cotação do dólar
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 2: PEGAR COTAÇÃO DO EURO

navegador.get("https://www.google.com/")

#xpath usar aspas simples

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3: Pegar a cotação do euro

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')

#codigo para mudar de virgula para ponto a divisão de casa decimal
cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)

navegador.quit()


#os valores atualizados

5.1082
5.539153293
312.26


In [55]:
# Passo 4: Importar base de dados e atualizar a base

import pandas as pd

tabela = pd.read_excel("C:/Users/mhgir/anaconda3/Projetos/Aula 3/Produtos.xlsx")

display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [56]:
# Passo 5 Recalcular os preços

# atualizar cotação dolar
# para comparar valores usamos dois iguais ==

# estou dizendo abaixo, localizar na tabela na coluna moeda onde está escrito Dólar e na coluna cotação destes casos mudar
# o valor para cotação _dolar. Está em float para transformar em valor
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# preço de compra = cotação * preço original

tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

#preço de venda = preço de compra * margem

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela ["Margem"]

display (tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.108200,5108.148918,1.40,7151.408485
1,Carro Renault,4500.00,Euro,5.539153,24926.189818,2.00,49852.379637
2,Notebook Dell,899.99,Dólar,5.108200,4597.328918,1.70,7815.459161
3,IPhone,799.00,Dólar,5.108200,4081.451800,1.70,6938.468060
4,Carro Fiat,3000.00,Euro,5.539153,16617.459879,1.90,31573.173770
5,Celular Xiaomi,480.48,Dólar,5.108200,2454.387936,2.00,4908.775872
6,Joia 20g,20.00,Ouro,312.260000,6245.200000,1.15,7181.980000


In [58]:
# Passo 6: Exportar a base atualizada

tabela.to_excel("C:/Users/mhgir/anaconda3/Projetos/Aula 3/Produtos_atualizados.xlsx")

#salvou um novo arquivo em excel

In [59]:
#Enviando email

!pip install pyautogui
import pyautogui
import pyperclip
import time

pyautogui.hotkey("ctrl","t")
pyperclip.copy("https://mail.google.com/mail/u/0/?tab=rm&ogbl#inbox")
pyautogui.hotkey("ctrl","v")
pyautogui.press("enter")

time.sleep(2)

pyautogui.click(x=137, y=264)
time.sleep(1)

pyautogui.write("matheushenriquegiraldi@gmail.com")
pyautogui.press("tab")
time.sleep(1)
pyautogui.press("tab")

pyperclip.copy("Cotações atualizadas")
pyautogui.hotkey("ctrl", "v")
pyautogui.press("tab")

texto = """"Prezados, bom dia

Segue planilha atualizada com a cotação do Dólar, Euro e Ouro.

Grato

"""

pyperclip.copy(texto)
pyautogui.hotkey("ctrl", "v")

pyautogui.click(x=1369, y=980)
time.sleep(1)
pyautogui.click(x=534, y=74)

pyperclip.copy("C:/Users/mhgir/anaconda3/Projetos/Aula 3")
pyautogui.hotkey("ctrl", "v", "enter")
pyautogui.click(x=391, y=407)
pyautogui.hotkey("enter")
time.sleep(2)

pyautogui.hotkey("ctrl", "enter")



In [51]:
import pyautogui
import pyperclip
import time


time.sleep(10)
print(pyautogui.position())

Point(x=391, y=407)
